Open the Python shell:

In [ ]:
python manage.py shell

### **Filtering** objects
To filter a QuerySet, you can use the ``filter()`` method of the manager. This method allows you to
specify the content of a SQL WHERE clause by using field lookups.
For example, you can use the following to filter Post objects by their title:

In [ ]:
>>> Post.objects.filter(title='Who was Django Reinhardt?')

In [ ]:
posts = Post.objects.filter(title='Who was Django Reinhardt?')
>>> print(posts.query)

By printing the query attribute of the QuerySet, we can get the SQL produced by it:

In [ ]:
SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", "blog_
post"."author_id", "blog_post"."body", "blog_post"."publish", "blog_
post"."created", "blog_post"."updated", "blog_post"."status" FROM "blog_post"
WHERE "blog_post"."title" = Who was Django Reinhardt? ORDER BY "blog_
post"."publish" DESC

The generated WHERE clause performs an exact match on the title column. The ORDER BY clause
specifies the default order defined in the ordering attribute of the Post model’s Meta options since
we haven’t provided any specific ordering in the QuerySet.  
Note that the query attribute is not part of the QuerySet public API.

### Using field lookups  
The previous QuerySet example consists of a filter lookup with an exact match. The QuerySet interface provides you with multiple lookup types.  
**Two underscores** are used to define the lookup type, with the format ``field__lookup``. For example, the following lookup produces an exact match:

In [ ]:
>>> Post.objects.filter(id__exact=1)

When no specific lookup type is provided, the lookup type is assumed to be exact. The following lookup is **equivalent to the previous one**:

In [ ]:
>>> Post.objects.filter(id=1)

You can generate a case-insensitive lookup with iexact:

In [ ]:
>>> Post.objects.filter(title__iexact='who was django reinhardt?')

You can also filter objects using a containment test. The ``contains`` lookup translates to a SQL lookup
using the ``LIKE`` operator:

In [ ]:
>>> Post.objects.filter(title__contains='Django')

The equivalent SQL clause is ``WHERE title LIKE`` '%Django%'. A case-insensitive version is also available,
named ``icontains``:

In [ ]:
>>> Post.objects.filter(title__icontains='django')

You can check for a given iterable (often a list, tuple, or another QuerySet object) with the ``in`` lookup.
The following example retrieves posts with an id that is 1 or 3:

In [ ]:
>>> Post.objects.filter(id__in=[1, 3])

The following example shows the **greater than** (``gt``) lookup:  
The equivalent SQL clause is ``WHERE ID > 3``

In [ ]:
>>> Post.objects.filter(id__gt=3)

This example shows the **greater than or equal to** lookup:

In [ ]:
>>> Post.objects.filter(id__gte=3)

This one shows the **less than** lookup:

In [ ]:
>>> Post.objects.filter(id__lt=3)

This shows the **less than or equal to** lookup:

In [ ]:
>>> Post.objects.filter(id__lte=3)

A **case-sensitive/insensitive starts-with** lookup can be performed with the ``startswith`` and ``istartswith``
lookup types, respectively:

In [ ]:
>>> Post.objects.filter(title__istartswith='who')

A **case-sensitive/insensitive ends-with** lookup can be performed with the ``endswith`` and ``iendswith``
lookup types, respectively:

In [ ]:
>>> Post.objects.filter(title__iendswith='reinhardt?')

#### Date
 An **exact** date lookup can be performed as
follows:

In [ ]:
>>> from datetime import date
>>> Post.objects.filter(publish__date=date(2024, 1, 31))

This shows how to filter a DateField or DateTimeField field **by year**:

In [ ]:
>>> Post.objects.filter(publish__year=2024)

You can also filter **by month**:

In [ ]:
>>> Post.objects.filter(publish__month=1)

And you can filter **by day**:

In [ ]:
>>> Post.objects.filter(publish__day=1)

You can chain additional lookups to date, year, month, and day. For example, here is a lookup for a **value greater than a given date**:

In [ ]:
>>> Post.objects.filter(publish__date__gt=date(2024, 1, 1))

To lookup ``related object fields``, you also use the **two-underscores notation**. For example, to retrieve
the posts written by the user with the admin username, use the following:

In [ ]:
>>> Post.objects.filter(author__username='admin')

You can also chain additional lookups for the related fields. For example, to retrieve posts written by any user with a username that starts with ad, use the following:

In [ ]:
>>> Post.objects.filter(author__username__starstwith='ad')

You can also filter by multiple fields. For example, the following QuerySet retrieves all posts published
in 2024 by the author with the username admin:

In [ ]:
>>> Post.objects.filter(publish__year=2024, author__username='admin')

### Chaining filters

The result of a filtered QuerySet is another QuerySet object. This allows you to chain QuerySets together.
You can build an equivalent QuerySet to the previous one by chaining multiple filters:

In [ ]:
>>> Post.objects.filter(publish__year=2024) \
>>> .filter(author__username='admin')

### Excluding objects
You can exclude certain results from your QuerySet by using the exclude() method of the manager.
For example, you can retrieve all posts published in 2024 whose titles don’t start with Why:

In [ ]:
>>> Post.objects.filter(publish__year=2024) \
>>> .exclude(title__startswith='Why')

### Ordering objects
The default order is defined in the ordering option of the model’s Meta. You can override the default
ordering using the order_by() method of the manager. For example, you can retrieve all objects
ordered by their title, as follows:

In [ ]:
>>> Post.objects.order_by('title')

Ascending order is implied. You can indicate descending order with a negative sign prefix, like this:

In [ ]:
>>> Post.objects.order_by('-title')

You can order by multiple fields. The following example orders objects by author first and then title:

In [ ]:
>>> Post.objects.order_by('author', 'title')

To order randomly, use the string '?', as follows:

In [ ]:
>>> Post.objects.order_by('?')

### Limiting QuerySets
You can limit a QuerySet to a certain number of results by using a subset of Python’s array-slicing
syntax. For example, the following QuerySet limits the results to 5 objects:

In [ ]:
>>> Post.objects.all()[:5]

This translates to a SQL LIMIT 5 clause. Note that negative indexing is not supported.

In [ ]:
>>> Post.objects.all()[3:6]

The preceding translates to a SQL OFFSET 3 LIMIT 6 clause, to return the fourth through sixth objects.
To retrieve a single object, you can use an index instead of a slice. For example, use the following to
retrieve the first object of posts in random order:

In [ ]:
>>> Post.objects.order_by('?')[0]

### Counting objects
The count() method counts the total number of objects matching the QuerySet and returns an integer.
This method translates to a SELECT COUNT(*) SQL statement. The following example returns the total
number of posts with an id lower than 3:

In [ ]:
>>> Post.objects.filter(id_lt=3).count()
2

### Checking if an object exists
The exists() method allows you to check if a QuerySet contains any results. This method returns
True if the QuerySet contains any items and False otherwise. For example, you can check if there are
any posts with a title that starts with Why using the following QuerySet:

In [ ]:
>>> Post.objects.filter(title__startswith='Why').exists()
False

### Deleting objects
If you want to delete an object, you can do it from an object instance using the delete() method, as
follows:

In [ ]:
>>> post = Post.objects.get(id=1)
>>> post.delete()

Note that deleting objects will also delete any dependent relationships for ForeignKey objects defined
with on_delete set to CASCADE.

### Complex lookups with Q objects
Field lookups using filter() are joined with a SQL AND operator. For example, filter(field1='foo ',
field2='bar') will retrieve objects where field1 is foo and field2 is bar. If you need to build more
complex queries, such as queries with OR statements, you can use Q objects.
A Q object allows you to encapsulate a collection of field lookups. You can compose statements by
combining Q objects with the & (and), | (or), and ^ (xor) operators.
For example, the following code retrieves posts with a title that starts with the string who or why
(case-insensitive):

In this case, we use the | operator to build an OR statement.

In [ ]:
>>> from django.db.models import Q
>>> starts_who = Q(title__istartswith='who')
>>> starts_why = Q(title__istartswith='why')
>>> Post.objects.filter(starts_who | starts_why)

You can read more about Q objects at https://docs.djangoproject.com/en/5.0/topics/db/queries/#complex-lookups-with-q-objects

### When QuerySets are evaluated
Creating a QuerySet doesn’t involve any database activity until it is evaluated. QuerySets will usually
return another unevaluated QuerySet. You can concatenate as many filters as you like to a QuerySet,
and you will not hit the database until the QuerySet is evaluated. When a QuerySet is evaluated, it
translates into a SQL query to the database.
QuerySets are only evaluated in the following cases:
• The first time you iterate over them
• When you slice them, for instance, Post.objects.all()[:3]
• When you pickle or cache them
• When you call repr() or len() on them
• When you explicitly call list() on them
• When you test them in a statement, such as bool(), or, and, or if